In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np

In [2]:


osprey_confirmed_nests = pd.read_excel('Osprey Data, Confirmed Nests as of 8.2.2023.xlsx', sheet_name='Confirmed Osprey Nests')

# Map data
map_data = gpd.read_file('MyWork/waterbudiees/NY_Waterbody_List.geojson')

osprey_confirmed_nests['geometry'] = gpd.points_from_xy(pd.to_numeric(osprey_confirmed_nests.faultlongitude, errors='coerce'), pd.to_numeric(osprey_confirmed_nests.faultlatitude, errors='coerce'))

osprey_confirmed_nests = gpd.GeoDataFrame(osprey_confirmed_nests, geometry=gpd.points_from_xy(osprey_confirmed_nests.faultlongitude, osprey_confirmed_nests.faultlatitude))

osprey_confirmed_nests = osprey_confirmed_nests.set_crs(epsg=4326)

# Ensure both GeoDataFrames have the same CRS
osprey_confirmed_nests = osprey_confirmed_nests.to_crs(epsg=3857)
map_data = map_data.to_crs(epsg=3857)

# Function to find distance from a nest to the nearest water body
def distance_to_nearest_water(nest, water_bodies):
    distances = water_bodies.geometry.distance(nest)
    return distances.min()

# Calculate distances for each nest
distances_to_water = osprey_confirmed_nests.geometry.apply(lambda x: distance_to_nearest_water(x, map_data))

# Calculate the average distance (in kilometers)
average_dist_to_water = distances_to_water.mean() / 1000

print(f"The average distance between confirmed nests and water bodies is approximately {average_dist_to_water:.2f} kilometers.")

The average distance between confirmed nests and water bodies is approximately 3.04 kilometers.


In [3]:
import geopandas as gpd
import pandas as pd
import numpy as np
from shapely.geometry import Point
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import MinMaxScaler
import folium
from folium.plugins import MarkerCluster
import haversine
from haversine import Unit
from math import radians, sin, cos, sqrt, atan2
from geopy.distance import geodesic
print("Packages imported successfully!")

Packages imported successfully!


In [4]:
osprey_nests = pd.read_excel('Osprey Data, Confirmed Nests as of 8.2.2023.xlsx', sheet_name='Outages, 2018-Current')

# Convert pandas DataFrame to GeoDataFrame
osprey_nests = gpd.GeoDataFrame(osprey_nests, geometry=gpd.points_from_xy(osprey_nests.faultlongitude, osprey_nests.faultlatitude))

outages_df = gpd.read_file('Outages01012020through09012023.csv')
map_data = gpd.read_file('MyWork/waterbudiees/NY_Waterbody_List.geojson')


In [5]:
map_data_CT = gpd.read_file('MyWork/waterbudiees/CT_Waterbody_List.geojson')
map_data_Maine = gpd.read_file('MyWork/waterbudiees/Maine_water.geojson')

In [6]:
grouped_outages = outages_df.groupby('OPCO')

# Create separate DataFrames for each group
outages_by_opco = {opco: group for opco, group in grouped_outages}

outages_nyseg = outages_df[outages_df.OPCO=='NYSEG']
outages_rge = outages_df[outages_df.OPCO=='RGE']
outages_cmp = outages_df[outages_df.OPCO=='CMP']
outages_ui = outages_df[outages_df.OPCO=='UI']

outages_cmp


,OPCO,Incident,IncSeqNo,OPCOIncStartNo,GroupNo,MalfStartDate,MalfStartTime,MalfEndDate,MalfEndTime,Duration,...,faultlongitude,ServiceCenter,ECC_OperatorLog,ThirdPartyAttacher,UI_TREE_CONTACT_TYPE_DESC,FaultFlocAlgoSource,DerivedFaultCircuit,WeatherIndex,iCDSOutageRemark,geometry
23,CMP,876687,1,672777,NULL,9/1/23,6:27:00,9/1/23,7:36:00,1.15,...,-70.299745,ECC,NULL,NULL,NULL,FuzzyDistPUB_Poles,416D1,22,NULL,None
25,CMP,876688,1,672784,NULL,9/1/23,8:12:00,9/1/23,8:56:00,0.7333,...,-69.781802,ECC,NULL,NULL,NULL,FuzzyDistPUB_Poles,240D1,17,NULL,None
27,CMP,876689,1,672783,NULL,9/1/23,8:09:00,9/1/23,9:00:00,0.85,...,-70.777591,ECC,NULL,NULL,NULL,FuzzyDistPUB_Poles,626D1,32,NULL,None
29,CMP,876690,1,672781,NULL,9/1/23,7:58:00,9/1/23,9:19:00,1.35,...,-69.2713,ECC,NULL,NULL,NULL,FuzzyDistPUB_Poles,837D1,22,NULL,None
31,CMP,876691,1,672788,NULL,9/1/23,8:37:00,9/1/23,9:58:00,1.35,...,-70.190537,ECC,NULL,NULL,NULL,FuzzyDistPUB_Poles,620D3,18,NULL,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177030,CMP,605147,0,108585,NULL,1/1/20,3:10:00,1/1/20,5:15:00,2.083333333,...,-69.52255,ECC,NULL,NULL,NULL,FuzzyDistPUB_Poles,219D2,47,NULL,None
177031,CMP,605148,0,108588,NULL,1/1/20,7:21:00,1/1/20,8:26:00,1.083333333,...,-70.76773,ECC,NULL,NULL,NULL,FuzzyDistPUB_Poles,661D1,28,NULL,None
177032,CMP,605149,0,108594,NULL,1/1/20,8:03:00,1/1/20,8:39:00,0.6,...,-69.951912,ECC,NULL,NULL,NULL,FuzzyDistPUB_Poles,256D2,35,NULL,None
177033,CMP,605150,0,108589,NULL,1/1/20,7:23:00,1/1/20,8:56:00,1.55,...,-70.787968,ECC,NULL,NULL,NULL,FuzzyDistPUB_Poles,602D3,27,NULL,None


In [9]:
# only choose columns 'CauseDesc', 'Division', 'CustomerCount', 'OPCO','faultlatitude','faultlongitude' from osprey_nests
osprey_nests = osprey_nests[['CauseDesc', 'Division', 'CustomerCount', 'OPCO','faultlatitude','faultlongitude']]
#  only choose columns 'OPCO', 'CustomerCount', CircuitVoltage', 'Division', 'CauseDesc', 'faultlatitude', 'faultlongitude' from outages_nyseg
outages_nyseg = outages_nyseg[['OPCO', 'CustomerCount', 'CircuitVoltage', 'Division', 'CauseDesc', 'faultlatitude', 'faultlongitude']]
#  only choose columns 'OPCO', 'CustomerCount', CircuitVoltage', 'Division', 'CauseDesc', 'faultlatitude', 'faultlongitude' from outages_rge
outages_rge = outages_rge[['OPCO', 'CustomerCount', 'CircuitVoltage', 'Division', 'CauseDesc', 'faultlatitude', 'faultlongitude']]


In [7]:
outages_ui = outages_ui[['OPCO', 'CustomerCount', 'CircuitVoltage', 'Division', 'CauseDesc', 'faultlatitude', 'faultlongitude']]

outages_cmp = outages_cmp[['OPCO', 'CustomerCount', 'CircuitVoltage', 'Division', 'CauseDesc', 'faultlatitude', 'faultlongitude']]


In [7]:
# Filter out records with missing longitude or latitude values
outages = outages_nyseg[(outages_nyseg['faultlongitude'] != '') & (outages_nyseg['faultlatitude'] != '')]
outages = outages_rge[(outages_rge['faultlongitude'] != '') & (outages_rge['faultlatitude'] != '')]
osprey_nests = osprey_nests[(osprey_nests['faultlongitude'] != '') & (osprey_nests['faultlatitude'] != '')]

# concatenate outages_nyseg and outages_rge
outages = pd.concat([outages_nyseg, outages_rge])




In [8]:
# Replace empty strings with NaN
outages_cmp['faultlongitude'] = outages_cmp['faultlongitude'].replace('', np.nan)
outages_cmp['faultlatitude'] = outages_cmp['faultlatitude'].replace('', np.nan)

outages_cmp['faultlongitude'] = outages_cmp['faultlongitude'].replace('NULL', np.nan)
outages_cmp['faultlatitude'] = outages_cmp['faultlatitude'].replace('NULL', np.nan)
# Drop rows with NaN values
outages_cmp = outages_cmp.dropna(subset=['faultlongitude', 'faultlatitude'])

# Convert to GeoDataFrame
outages_cmp = gpd.GeoDataFrame(outages_cmp, geometry=gpd.points_from_xy(outages_cmp.faultlongitude.astype(float), outages_cmp.faultlatitude.astype(float)))


/var/folders/t2/277s42mj7ydg5b4lq0dd24jc0000gn/T/ipykernel_62864/1554872296.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  outages_cmp['faultlongitude'] = outages_cmp['faultlongitude'].replace('', np.nan)
/var/folders/t2/277s42mj7ydg5b4lq0dd24jc0000gn/T/ipykernel_62864/1554872296.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  outages_cmp['faultlatitude'] = outages_cmp['faultlatitude'].replace('', np.nan)
/var/folders/t2/277s42mj7ydg5b4lq0dd24jc0000gn/T/ipykernel_62864/1554872296.py:5: SettingWithC

In [9]:
map_data_Maine.head()
map_data_Maine.shape

(6399, 7)

In [10]:
map_data_Maine.columns

Index(['OBJECTID', 'IWWH_ID', 'RATING', 'GlobalID', 'SHAPE_Length',
       'SHAPE_Area', 'geometry'],
      dtype='object')

In [9]:
# from shapely.geometry import Point
# import geopandas as gpd

# # Convert 'faultlongitude' and 'faultlatitude' to numeric type
# outages['faultlongitude'] = pd.to_numeric(outages['faultlongitude'], errors='coerce')
# outages['faultlatitude'] = pd.to_numeric(outages['faultlatitude'], errors='coerce')

# # Drop rows where latitude or longitude is NaN
# outages = outages.dropna(subset=['faultlatitude', 'faultlongitude'])

# # Create the 'geometry' column
# outages['geometry'] = outages.apply(lambda row: Point(row['faultlongitude'], row['faultlatitude']), axis=1)

# # Assuming 'map_data' is a GeoDataFrame with a 'geometry' column
# map_data_sindex = map_data.sindex

# # Define a distance threshold (adjust as needed)
# distance_threshold = 3.04

# # Create empty columns to store results
# outages['nearest_waterbody_idx'] = None
# outages['nearest_waterbody_distance'] = None

# # Iterate over each row and update the columns
# for index, row in outages.iterrows():
#     # Calculate the bounding box of the pole's geometry
#     pole_bbox = row['geometry'].bounds
    
#     # Use the bounding box to query the spatial index for potential nearby water bodies
#     potential_waterbodies_index = map_data_sindex.intersection(pole_bbox)
    
#     min_distance = float('inf')
#     nearest_waterbody_idx = None
    
#     # Iterate over potential nearby water bodies
#     for waterbody_index in potential_waterbodies_index:
#         waterbody = map_data.iloc[waterbody_index].geometry
        
#         # Perform a bounding box check to exclude distant water bodies
#         if row['geometry'].distance(waterbody) < distance_threshold:
#             # Calculate the distance and update if it's closer than the previous
#             distance_to_water = row['geometry'].distance(waterbody)
#             if distance_to_water < min_distance:
#                 min_distance = distance_to_water
#                 nearest_waterbody_idx = waterbody_index
    
#     # Update the columns
#     outages.at[index, 'nearest_waterbody_idx'] = nearest_waterbody_idx
    
#     # Handle the case where no valid water body was found within the threshold
#     if min_distance == float('inf'):
#         outages.at[index, 'nearest_waterbody_distance'] = None  # or any other suitable placeholder
#     else:
#         outages.at[index, 'nearest_waterbody_distance'] = min_distance

In [11]:
from sklearn.metrics.pairwise import euclidean_distances

outages_cmp['faultlongitude'] = pd.to_numeric(outages_cmp['faultlongitude'], errors='coerce')
outages_cmp['faultlatitude'] = pd.to_numeric(outages_cmp['faultlatitude'], errors='coerce')

outages_cmp = outages_cmp.dropna(subset=['faultlatitude', 'faultlongitude'])

outages_cmp['geometry'] = outages_cmp.apply(lambda row: Point(row['faultlongitude'], row['faultlatitude']), axis=1)

waterbody_coordinates = map_data_Maine['geometry'].apply(lambda x: [x.centroid.x, x.centroid.y]).to_list()

waterbody_coordinates_array = np.array(waterbody_coordinates)

outages_coordinates = outages_cmp['geometry'].apply(lambda x: [x.x, x.y]).to_list()

outages_coordinates_array = np.array(outages_coordinates)

distances = euclidean_distances(outages_coordinates_array, waterbody_coordinates_array)

nearest_waterbody_idx = np.argmin(distances, axis=1)
nearest_waterbody_distance = np.min(distances, axis=1)
# nearest_waterbody_info = map_data_CT['NAME'].loc[nearest_waterbody_idx]


# Assign the results to the 'outages' DataFrame
# outages_ui['nearest_waterbody_name'] = nearest_waterbody_info.values
outages_cmp['nearest_waterbody_idx'] = nearest_waterbody_idx
outages_cmp['nearest_waterbody_distance'] = nearest_waterbody_distance

In [12]:
outages_cmp.head()

,OPCO,CustomerCount,CircuitVoltage,Division,CauseDesc,faultlatitude,faultlongitude,geometry,nearest_waterbody_idx,nearest_waterbody_distance
23,CMP,12,12470,Lewiston,Cause Unknown/Undetermined,43.881638,-70.299745,POINT (-70.29975 43.88164),178,0.024962
25,CMP,4,12470,Augusta,Animal Squirrel Contact,44.096295,-69.781802,POINT (-69.78180 44.09629),5700,0.021982
27,CMP,86,12470,Alfred,Defective Line Cutout,43.424700,-70.777591,POINT (-70.77759 43.42470),799,0.019104
29,CMP,40,12470,Dover,Tree on Conductor Tree In ROW,44.863163,-69.271300,POINT (-69.27130 44.86316),540,0.026334
31,CMP,9,12470,Brunswick,Animal Squirrel Contact,43.802775,-70.190537,POINT (-70.19054 43.80277),195,0.074633


In [17]:
# outages.head()

In [13]:
crs_info = map_data_Maine.crs
crs_info

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [14]:
outages_cmp.crs

In [15]:
outages_cmp.dtypes

OPCO                            object
CustomerCount                   object
CircuitVoltage                  object
Division                        object
CauseDesc                       object
faultlatitude                  float64
faultlongitude                 float64
geometry                      geometry
nearest_waterbody_idx            int64
nearest_waterbody_distance     float64
dtype: object

In [16]:
conversion_factor = 111

outages_cmp['nearest_waterbody_distance_km'] = outages_cmp['nearest_waterbody_distance'] * conversion_factor

In [17]:
outages_cmp.head()

,OPCO,CustomerCount,CircuitVoltage,Division,CauseDesc,faultlatitude,faultlongitude,geometry,nearest_waterbody_idx,nearest_waterbody_distance,nearest_waterbody_distance_km
23,CMP,12,12470,Lewiston,Cause Unknown/Undetermined,43.881638,-70.299745,POINT (-70.29975 43.88164),178,0.024962,2.770799
25,CMP,4,12470,Augusta,Animal Squirrel Contact,44.096295,-69.781802,POINT (-69.78180 44.09629),5700,0.021982,2.440012
27,CMP,86,12470,Alfred,Defective Line Cutout,43.424700,-70.777591,POINT (-70.77759 43.42470),799,0.019104,2.120574
29,CMP,40,12470,Dover,Tree on Conductor Tree In ROW,44.863163,-69.271300,POINT (-69.27130 44.86316),540,0.026334,2.923080
31,CMP,9,12470,Brunswick,Animal Squirrel Contact,43.802775,-70.190537,POINT (-70.19054 43.80277),195,0.074633,8.284261


In [18]:
outages_cmp = outages_cmp.drop(['nearest_waterbody_distance'],axis=1)

In [19]:
outages_cmp.head()

,OPCO,CustomerCount,CircuitVoltage,Division,CauseDesc,faultlatitude,faultlongitude,geometry,nearest_waterbody_idx,nearest_waterbody_distance_km
23,CMP,12,12470,Lewiston,Cause Unknown/Undetermined,43.881638,-70.299745,POINT (-70.29975 43.88164),178,2.770799
25,CMP,4,12470,Augusta,Animal Squirrel Contact,44.096295,-69.781802,POINT (-69.78180 44.09629),5700,2.440012
27,CMP,86,12470,Alfred,Defective Line Cutout,43.424700,-70.777591,POINT (-70.77759 43.42470),799,2.120574
29,CMP,40,12470,Dover,Tree on Conductor Tree In ROW,44.863163,-69.271300,POINT (-69.27130 44.86316),540,2.923080
31,CMP,9,12470,Brunswick,Animal Squirrel Contact,43.802775,-70.190537,POINT (-70.19054 43.80277),195,8.284261


In [20]:
outages_cmp.to_csv('outages_with_waterbody_distances_CMP.csv')

In [3]:
import geopandas as gpd
import folium

# Create a folium map centered around the mean of your water bodies' latitudes and longitudes
m = folium.Map(location=[map_data_CT['geometry'].centroid.y.mean(), map_data_CT['geometry'].centroid.x.mean()], zoom_start=10)

# Add water bodies to the map
for _, row in map_data_CT.iterrows():
    folium.GeoJson(row['geometry']).add_to(m)

# Save the map
m.save("water_bodies_map.html")

/var/folders/t2/277s42mj7ydg5b4lq0dd24jc0000gn/T/ipykernel_57354/2941740214.py:5: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  m = folium.Map(location=[map_data_CT['geometry'].centroid.y.mean(), map_data_CT['geometry'].centroid.x.mean()], zoom_start=10)


In [4]:
m = folium.Map(location=[map_data_Maine['geometry'].centroid.y.mean(), map_data_Maine['geometry'].centroid.x.mean()], zoom_start=10)

# Add water bodies to the map
for _, row in map_data_Maine.iterrows():
    folium.GeoJson(row['geometry']).add_to(m)

# Save the map
m.save("water_bodies_map_maine.html")

/var/folders/t2/277s42mj7ydg5b4lq0dd24jc0000gn/T/ipykernel_57354/2577520153.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  m = folium.Map(location=[map_data_Maine['geometry'].centroid.y.mean(), map_data_Maine['geometry'].centroid.x.mean()], zoom_start=10)
